# III. Pares candidatos y Autores similares

En esta parte idetificaremos pares candidatos de tweets y aplicaremos thresholding para definir autores similares.

In [4]:
import numpy as np
import pandas as pd
import collections
import itertools
import ast
import random

### Importar Trabajo previo

In [5]:
resumen = pd.read_csv('preprocesado.csv', converters={"shingles": ast.literal_eval})

Consolidar FHs a partir de los archivos guardados en disco.

In [6]:
FH = np.genfromtxt('fhs/file0.csv', delimiter=',')
for i in range(1, 10):
    FH2 = np.genfromtxt(f'fhs/file{i}.csv', delimiter=',')
    FH = np.concatenate((FH, FH2), axis=1)
FH.shape

(20, 203726)

## Banding

Mostrar distancia mínima a la que debe estar dos tweet para ser considerados pares candidatos

In [7]:
r = 10
b = 2
round((1/b)**(1/r), 3)

0.933

Ejecutar banding y obtener pares candidatos

In [8]:
# https://towardsdatascience.com/locality-sensitive-hashing-how-to-find-similar-items-in-a-large-set-with-precision-d907c52b05fc
sig_mat = FH
n, d = sig_mat.shape
hashbuckets = collections.defaultdict(set)
bands = np.array_split(sig_mat, b, axis=0)
for i,band in enumerate(bands):
    for j in range(d):
        # The last value must be made a string, to prevent accidental
        # key collisions of r+1 integers when we really only want
        # keys of r integers plus a band index
        band_id = tuple(list(band[:,j])+[str(i)])
        hashbuckets[band_id].add(j)
candidate_pairs = set()
for bucket in hashbuckets.values():
    if len(bucket) > 1:
        for pair in itertools.combinations(bucket, 2):
            candidate_pairs.add(pair)
print(f"Obtuvimos {len(candidate_pairs)} pares candidatos, con b = {b} y r = {r}")

Obtuvimos 250425 pares candidatos, con b = 2 y r = 10


## Limpiar pares de candidatos

Quitar tweets idénticos de los pares similares

In [9]:
trimmed_cadidate_pairs = []
for pair in candidate_pairs:
        a, b = pair
        text_a = resumen.text[a]
        text_b = resumen.text[b]
        if (text_a != text_b) and ('http' not in text_a) and ('http' not in text_b):
                trimmed_cadidate_pairs.append((a, b))
print(f"Quedan {len(trimmed_cadidate_pairs)} pares candidatos")

Quedan 107584 pares candidatos


Mostrar 3 pares de tweets presuntamente similares

In [10]:
for pair in random.sample(list(trimmed_cadidate_pairs), 3):
    a, b = pair
    print(f"1. {resumen.screen_name[a]}: ",resumen.text[a])
    print(f"2. {resumen.screen_name[b]}: ",resumen.text[b])
    print("----------")

1. FarizoGonzales:  @Jaime_Bassa YA TERMINASTE TU TRABAJO O TE PAGAN PARA HACER CAMPAÑA?
ES𝐎  SE LLAMA 
𝐑𝐎𝐁𝐎
𝐅𝐑𝐄𝐒𝐂𝐔𝐑𝐀
𝐎𝐏𝐎𝐑𝐓𝐔𝐍𝐈𝐒𝐌𝐎
𝐑𝐄𝐈𝐑𝐓𝐄 𝐃𝐄𝐋.𝐏𝐔𝐄𝐁𝐋𝐎
𝐅𝐃𝐑
2. magiavioc:  @Jaime_Bassa HAGAN LO QUE HAGAN   DIGAN LO QUE  DIGAN   IGUAL SE IRAN DE  PATA EN EL CULO
----------
1. Araneddstr:  @Sinfiltros_tv @PanchoOrregoG @fernando_atria @fernando_atria va @PanchoOrregoG ponganle fecha.
2. cristovich:  @jhonnygualtieri @Sinfiltros_tv @PanchoOrregoG @fernando_atria Las águilas no cazan moscas
----------
1. EsteOtro22:  @patriciapolitz @convencioncl @rkatrileo @MillaburAdolfo @ElisaLoncon @NoNeutrales IGNORANTE CTM 🌳
2. maggiescobarC:  @nenerepublicana @patriciapolitz @convencioncl @rkatrileo @MillaburAdolfo @ElisaLoncon @NoNeutrales 👍👍👍
----------


## Identificar autores similares

Contar cuantos tweets similares hay para cada par de autores considerados candidatos similares

In [11]:
count_dict = dict()

def def_value():
    return 0
for pair in trimmed_cadidate_pairs:
    a, b = pair
    user_a = resumen.screen_name[a]
    user_b = resumen.screen_name[b]
    if user_a != user_b:
        try:
            count_dict[user_a][user_b] += 1
        except KeyError:
            count_dict[user_a] = collections.defaultdict(def_value) # No retorna key error
            count_dict[user_a][user_b] += 1
            
        try:
            count_dict[user_b][user_a] += 1
        except KeyError:
            count_dict[user_b] = collections.defaultdict(def_value) # No retorna key error
            count_dict[user_b][user_a] += 1
            

Mostrar solo los autores que están por sobre un threshold de tweets similares

In [21]:
count_df = pd.DataFrame(count_dict).reset_index()
count_long = pd.melt(count_df, id_vars='index')
similar_authors = count_long[count_long.value >= 25].rename({'index': 'user_1', 'variable': 'user_2', 'value':'sim_tweet_count'}, axis = 1)
similar_authors.sort_values("sim_tweet_count", ascending=False)

,user_1,user_2,sim_tweet_count
1607327,osotroncoso,x1educalidad,131.0
2802006,x1educalidad,osotroncoso,131.0
2801289,malahierba84,osotroncoso,105.0
2798119,osotroncoso,malahierba84,105.0
1797811,MarVeraOjeda9,Pedro_Pablo_74,96.0
831787,Pedro_Pablo_74,MarVeraOjeda9,96.0
2791052,exevalpo,Cristia56264488,96.0
2795718,Cristia56264488,exevalpo,96.0
2621142,Pedro_Pablo_74,LusalguaJames,77.0
1797813,LusalguaJames,Pedro_Pablo_74,77.0


In [22]:
similar_authors.shape

(40, 3)

Función para imprimir tweets de dos candidatos pares:

In [23]:
def show_tweets_from_candidates(user_1, user_2, amount):
    iterations = 0
    for pair in trimmed_cadidate_pairs:
        a, b = pair
        user_a = resumen.screen_name[a]
        user_b = resumen.screen_name[b]
        if ((user_a == user_1) and (user_b == user_2)) or ((user_b == user_1) and (user_a == user_2)):
            print("By user ", user_a,":\n",  f'"{resumen.text[a]}"\n')
            print("By user ", user_b, ":\n", f'"{resumen.text[b]}"')
            print("_______________________________________________________\n")
            if iterations == (amount-1):
                break
            iterations+=1

Veamos tres ejemplos de pares de tweets de un par de autores similares.

In [17]:
show_tweets_from_candidates("Natacha55538315", "MAJOCAPA196556", 3)

A continuación podemos examinar un cierto número de tweets para un n dado de autores similares.

In [51]:
def shown_n_similar_authors_examples(n_author_pairs, n_examples_per_pair):
    iteration = 0
    for index, row in similar_authors.iterrows():
        user_1 = row.user_1
        user_2 = row.user_2
        show_tweets_from_candidates(user_1, user_2, n_examples_per_pair) # Sacar 3 ejemplos por cada par
        if iteration == (n_author_pairs-1):
            break
        iteration += 1
shown_n_similar_authors_examples(2, 3)
# Se mostraran 3 ejemplos para 2 pares de autores similares: 

By user  aurora4310 :
 "@CamiRT7 @AprendizCrypto @pillan197811 @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @danielstingo @MEQChile Se pico"

By user  JaggerGuti :
 "@Elinis_Benetash @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @danielstingo @MEQChile @GAMBA_CL O empiezo yo? 😂"
_______________________________________________________

By user  JaggerGuti :
 "@Elinis_Benetash @sotan_n @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @danielstingo @MEQChile Propiedad y digna... 😂 😂 😂"

By user  aurora4310 :
 "@AprendizCrypto @pillan197811 @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @danielstingo @MEQChile No tienen ningún poder"
_______________________________________________________

By user  JaggerGuti :
 "@Elinis_Benetash @sotan_n @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @danielstingo @MEQChile Propiedad y digna... 😂 😂 😂"

By user  aurora4310 :
 "@AprendizCrypto @pillan197811 @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @dani